[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

**ADVERTENCIA:**  

Para poder realizar exitosamente los ejercicios de esta notebook, es necesario haber seguido al pie de la letra y en orden sucesivo las instrucciones de todas las notebooks previas.

https://docs.djangoproject.com/en/2.1/topics/forms/

## Formas en HTML.

Las formas en HTML se definene mediante la etiquete *&tl;form&gt;* de la siguiente forma:

```
<form action="una acción" method="método">
...
...
</form>
```
Las formas fueron creadas para interactuar con el usuario obteniendo información y/o ejecutando una acción a partir de elementos tipo *&lt;input&gt;*.

Las acciones pueden:
* Enviar la información recabada a una URL específica.
   * El método HTTP de envío de la información puede ser *GET* o *POST*.
   * O por medio de la ejecución de una función de Javascript.

Para mayor referencia consultar en https://www.w3schools.com/html/html_forms.asp

## Elementos de una forma en *Django*.

Django cuenta con un manejo muy avanzado de formas definiendo los siguientes elementos.
 
* Formas.
* Campos (field).
* Widgets.
* Validadores.

## El archivo *forms.py*.

Por convención, el archivo *forms.py* contiene las definiciones de formas de una aplicación.

## La clase *django.forms.Form*.

La clase *django.forms.Form* es el elemento principal de una forma en *Django*. 

Un objeto instanciado de esta clase se puede contener atributos muy similares a los definidos en los modelos, los cuales se conocen como campos (fields).

https://docs.djangoproject.com/en/2.1/ref/forms/api/#django.forms.Form

## La clase *django.forms.ModelForm*.

https://docs.djangoproject.com/en/2.1/topics/forms/modelforms/#django.forms.ModelForm

## Los campos.

https://docs.djangoproject.com/en/2.1/ref/forms/fields/#built-in-field-classes

## Los *widgets* de los campos.

https://docs.djangoproject.com/en/2.1/ref/forms/widgets/

In [1]:
from django.forms import Widget

## Validadores.
https://docs.djangoproject.com/en/2.1/ref/validators/

## Protección contra el envío de falsificación de peticiones entre sitios (CSRF).

Cross Site Request Forgery.

https://docs.djangoproject.com/en/2.1/ref/csrf/

django.middleware.csrf.CsrfViewMiddleware

### La etiqueta de plantilla *csrf*.

```
{% csrf_token %}
```


In [2]:
from tutorial.tutorial import settings as settings

In [3]:
settings.MIDDLEWARE

['django.middleware.security.SecurityMiddleware',
 'django.contrib.sessions.middleware.SessionMiddleware',
 'django.middleware.common.CommonMiddleware',
 'django.middleware.csrf.CsrfViewMiddleware',
 'django.contrib.auth.middleware.AuthenticationMiddleware',
 'django.contrib.messages.middleware.MessageMiddleware',
 'django.middleware.clickjacking.XFrameOptionsMiddleware']

## La función *django.shortcuts.render()*

```
render(request, '<plantilla>', context=<contexto>)
```

## Creación de una forma que consume una API REST para realizar altas.

### Definición de la ruta para la forma.

In [4]:
%cp src/20/urls.py tutorial/api/urls.py

In [5]:
%cat tutorial/api/urls.py

from django.urls import path, re_path
from . import views, endpoint_views, template_views

urlpatterns = [path('', views.vista),
               path('carga', views.carga),
               re_path(r'^(?P<clave>[0-9]{4}$)', endpoint_views.clave),
               path('vista/', template_views.vista),
               path('valida/', template_views.valida),
               path('alta/', template_views.forma),
         ]

### Definción de una forma.

In [6]:
%cp src/20/forms.py tutorial/api/forms.py

In [7]:
%cat tutorial/api/forms.py

from django import forms

CARRERAS =(('Sistemas', 'Sistemas'), 
           ('Derecho', 'Derecho'), 
           ('Actuaría', 'Actuaría'),
           ('Arquitectura', 'Arquitectura'),
           ('Administración', 'Administración'))

class FormaAlumno(forms.Form):
    numero_de_cuenta = forms.IntegerField(label='Número de cuenta',
                                          min_value=1000, 
                                          max_value=9999,
                                          error_messages={'required': 'Dato requerido'})
    nombre = forms.CharField(max_length=50, 
                             label='Nombre')
    primer_apellido = forms.CharField(max_length=50, 
                                      label='Primer apellido')
    segundo_apellido = forms.CharField(max_length=50, 
                                       label='Segundo apellido', 
                                       required=False)
    carrera = forms.ChoiceField(label='Carrera', 
                              

### La función de vista que contiene la forma.

In [8]:
%cp src/20/template_views.py tutorial/api/template_views.py

In [9]:
%cat tutorial/api/template_views.py

from .models import Alumno
from .forms import FormaAlumno
from django.http import HttpResponseRedirect, HttpResponse, JsonResponse
from django.shortcuts import render, render_to_response
from requests import post

campos = ('numero_de_cuenta', 'nombre', 'primer_apellido', 'segundo_apellido', 'carrera', 'semestre', 'promedio', 'al_corriente')


def vista(request):
    lista = [[(campo, getattr(alumno, campo)) for campo in campos] for alumno in Alumno.objects.all()]
    return render_to_response('listado.html',{'lista': lista}) 


def valida(request):
    lista = [[getattr(alumno, campo) for campo in campos] for alumno in Alumno.objects.all()]
    return render_to_response('valida.html',{'lista': lista}) 

def forma(request):
    if request.method == 'POST':
        forma = FormaAlumno(request.POST)
        if forma.is_valid():
            datos = request.POST.dict()
            '''
            datos.pop('csrfmiddlewaretoken')
            cuenta = datos.pop('numero_de_cuenta')
          

### La plantilla que incluye a la forma.

In [10]:
%cp src/20/forma.html tutorial/templates/forma.html

In [11]:
%cat tutorial/templates/forma.html

{% extends "base.html" %}
{% block encabezado %}Forma de alta {% endblock %}
{% block cuerpo %}
    <form action="/api/alta/" method="post">
        <div class="form-group">
            {% csrf_token %}
            {{ forma }}
        </div>
        <input type="submit" value="Enviar">
    </form>
{% endblock %}

* Se iniciará el servidor.

In [12]:
%cd tutorial

/opt/pythonista/py231/tutorial


**ADVERTENCIAS:** 

* Al ejecutar la siguiente celda el servidor se inciará desde la notebook, por lo que para ejecutar cualquier otra celda es necesario interrumpir la ejecución del kernel.

* Asegúrese que no haya otro servicio escuchando en el puerto *5000*.

In [13]:
!./manage.py runserver 0.0.0.0:5000

Performing system checks...

System check identified no issues (0 silenced).
February 28, 2019 - 22:43:47
Django version 2.1.7, using settings 'tutorial.settings'
Starting development server at http://0.0.0.0:5000/
Quit the server with CONTROL-C.
[28/Feb/2019 22:43:58] "GET /api/alta/ HTTP/1.1" 200 2123
[28/Feb/2019 22:44:13] "POST /api/alta/ HTTP/1.1" 200 256
[28/Feb/2019 22:44:53] "GET /api/alta/ HTTP/1.1" 200 2123
[28/Feb/2019 22:45:12] "POST /api/alta/ HTTP/1.1" 200 277
^C


http://localhost:5000/api/alta/

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>